****
# Algorithmic Problems solved in Python 3.7 (cont.)
****
<p style="text-align: right"><i>Jesus Perez Colino<br>First version: November 2018<br></i></p>

## About this notebook: 
****
Notebook prepared by **Jesus Perez Colino** Version 0.2, First Released: 25/11/2018, Alpha

- This work is licensed under a [Creative Commons Attribution-ShareAlike 3.0 Unported License](http://creativecommons.org/licenses/by-sa/3.0/deed.en_US). This work is offered for free, with the hope that it will be useful.


- **Summary**: This notebook is a continuation of the previous notebooks in algorithmic problems, but this time for Python 3.7. Some of them are well-know in Computer Science (searching, sorting...), and other are more functional programming oriented or mathematically based. It includes examples of list comprenhension, map/reduce functions, generators, decorators, HPC simulations...


- **Index**:
    - Problem 1: Is Fibonacci? (...because is tradition) 
    - Problem 2: Charles Babbage's homage
    - Problem 3: Dynamic programming 
    - Problem 4: Knapsack problem (another classical)
    - Problem 5: Customizing String Formatting
    - Problem 6: Implementing a Data Model or Type System using Descriptors
    - Problem 7: Calling a Method on an Object given the Name as a string

In [1]:
import IPython
import numpy as np
np.random.seed(12345)
import pandas as pd
import matplotlib.pyplot as plt
import watermark

import warnings
warnings.filterwarnings('ignore')

%load_ext watermark
%matplotlib inline

print(' Reproducibility conditions for this notebook '.center(85,'-'))
%watermark -n -v -m -p numpy,scipy,matplotlib,pandas
print('-'*85)

-------------------- Reproducibility conditions for this notebook -------------------
Sun Dec 01 2019 

CPython 3.7.3
IPython 7.6.1

numpy 1.16.4
scipy 1.2.1
matplotlib 3.1.0
pandas 0.24.2

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 94 Stepping 3, GenuineIntel
CPU cores  : 8
interpreter: 64bit
-------------------------------------------------------------------------------------


## Problem 1: Is Fibonacci?

Given a list of integers `mylist`, write a program to determine if any of the elements in `mylist` is an element of the Fibonacci sequence.

In [21]:
def solve(mylist):
    fibo_list = [0, 1]
    next_fib = sum(fibo_list[-2:])
    while next_fib < max(mylist)+1:
        fibo_list.append(next_fib)
        next_fib =  sum(fibo_list[-2:])
    for element in mylist:
        if element in fibo_list: print('{} is in the Fib. sequence'.format(element))
        else: print('{} is not in the Fib. sequence'.format(element))
    return

In [22]:
my_list = [2,3,4,5,6,7,8]
solve(my_list)

2 is in the Fib. sequence
3 is in the Fib. sequence
4 is not in the Fib. sequence
5 is in the Fib. sequence
6 is not in the Fib. sequence
7 is not in the Fib. sequence
8 is in the Fib. sequence


## Problem 2: Charles Babbage's homage

Charles Babbage (1791 - 1871) loooking ahead to the sorts of problems his Analytical Engine would be able to solve, gave this example:
> What's the smallest positive integer whose square ends in the digits 269,696?

He thought the answer might be 99,736, whose square is 9,947,269,696 but he could not be certain...

In [16]:
x = [x for x in range(100000) if (x*x) % 1000000 == 269696][0]
print(x, 'squared is equal to', x*x, 'and it ends in ', str(x*x)[-6:])

25264 squared is equal to 638269696 and it ends in  269696


In [75]:
x = [x for x in range(100000) if str(x*x)[-6:]== '269696'][0]
print(x, 'squared is equal to', x*x, 'and it ends in ', str(x*x)[-6:])

25264 squared is equal to 638269696 and it ends in  269696


## Problem 3: Dynamic programming

Find the number of sets of integers in `[ 2, 4, 6, 10 ]` that add up to `16`

In [53]:
# First solution: recursive version (no optimal)

def recursive(array, total, i): 
    if total == 0 : 
        return 1
    elif total < 0 : 
        return 0
    elif i < 0 : 
        return 0
    elif total < array[i]: 
        return recursive(array, total, i-1)
    else: 
        return recursive(array, total - array[i], i-1) + recursive(array, total, i-1)
    

def count_set(array, total):
    return recursive(array, total, len(array)-1)


In [56]:
myarray = [2,4,6,10]
total_sum = 16

In [63]:

count_set(myarray, total_sum)

2

In [50]:
# Second solution: dynamic programming solution (memoizing)

def count_sets_dp(array, total):
    mem = {}
    return dp(array, total, len(array)-1, mem)

def dp(array, total, i, mem):
    key = str(total) + ':' + str(i)
    if key in mem:
        return mem[key]
    elif total == 0:
        return 1
    elif total < 0:
        return 0
    elif i < 0:
        return 0
    elif total < array[i]:
        to_return = dp(array, total, i-1, mem)
    else: 
        to_return = dp(array, total-array[i], i-1, mem) \
                    + dp(array, total, i-1, mem)
        
    mem[key] = to_return
    return to_return


In [62]:

count_sets_dp(a, total_sum)    


2

## Problem 4: Knapsack problem

Given a set of items, each with a weight and a value, determine the number of each item to include in a collection, so that the total weight is less or equal to a given limit and the total value is as larger as possible.

In [17]:
# list of items: tuple of tuples of ('name', weight, value)

items = (
    ("map", 9, 150), ("compass", 13, 35), ("water", 153, 200), ("sandwich", 50, 160),
    ("glucose", 15, 60), ("tin", 68, 45), ("banana", 27, 60), ("apple", 39, 40),
    ("cheese", 23, 30), ("beer", 52, 10), ("suntan cream", 11, 70), ("camera", 32, 30),
    ("t-shirt", 24, 15), ("trousers", 48, 10), ("umbrella", 73, 40),
    ("waterproof trousers", 42, 70), ("waterproof overclothes", 43, 75),
    ("note-case", 22, 80), ("sunglasses", 7, 20), ("towel", 18, 12),
    ("socks", 4, 50), ("book", 30, 10),
    )

max_weight = 400

In [18]:
def total_value(items, max_weight):
    return sum([x[2] for x in items]) if sum([x[1] for x in items])<max_weight else 0

In [45]:
# Recursive dynamic algorithmic solution

cache = {}
def solve(items, max_weight):
    if not items:
        return()
    if (items, max_weight) not in cache:
        head = items[0]
        tail = items[1:]
        include = (head,) + solve(tail, max_weight - head[1])
        dont_include = solve(tail, max_weight)
        if total_value(include, max_weight) > total_value(dont_include, max_weight):
            answer = include
        else:
            answer = dont_include
        cache[(items, max_weight)] = answer
    return cache[(items, max_weight)]



In [47]:
cache = {}
solution = solve(items, max_weight)

print('items: ')
for x in solution: print(x[0])
print('='*20)
print('value: ', total_value(solution, max_weight))
print('weight: ', sum([x[1] for x in solution]))
print('='*20)

items: 
map
compass
water
sandwich
glucose
banana
suntan cream
waterproof trousers
waterproof overclothes
note-case
sunglasses
socks
value:  1030
weight:  396


## Problem 5: Customizing String Formatting
Build a Date class, with year, month and day as inputs, that is able to return the date in multiple formats, using an external format dictionary.

In [62]:
_formats = {
    'ymd' : '{d.year}--{d.month}--{d.day}',
    'mdy' : '{d.month}//{d.day}//{d.year}',
    'dmy' : '{d.day}||{d.month}||{d.year}'
    }

In [63]:
class Date(object):
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)
    def __str__(self):
        return format(d,'ymd')

In [64]:
d = Date(2012, 12, 21)
print(d)

2012--12--21


In [65]:
f"The date is {format(d)}"

'The date is 2012--12--21'

In [66]:
f"The date is {format(d, 'mdy')}"

'The date is 12//21//2012'

In [67]:
f"The date is {format(d, 'dmy')}"


'The date is 21||12||2012'

## Problem 6: Implementing a Data Model or Type System using Descriptors

You want to define several kinds of data structures, but want to enforce constraints on the values that are allowed to be assigned to certain attributes.

In [88]:
# Base class. Uses a descriptor to set a value

class Descriptor(object):
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

In [169]:
# Descriptor for enforcing types

class Typed(Descriptor):
    expected_type = type(None)
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected ' + str(self.expected_type))
        super().__set__(instance, value)

In [90]:
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super().__set__(instance, value)

In [91]:
class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super().__init__(name, **opts)
    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, value)

These classes should be viewed as basic building blocks from which you construct a data model or type system. 

Continuing, here is some code that implements some different kinds of data:

In [92]:
class Integer(Typed):
    expected_type = int

class UnsignedInteger(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float

class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str

class SizedString(String, MaxSized):
    pass

In [158]:
class Stock(list):
    # Specify constraints
    name = SizedString('name',size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
    def __str__(self):
        return f"{[self.name, self.shares, self.price]}"


In [162]:
s = Stock('GOOG', 10, 11.1)
print(s)


['GOOG', 10, 11.1]


In [167]:
s = Stock(11.1, 11.1, 11.1)

print(s) # <<<--- It should print an Type ERROR 

TypeError: expected <class 'str'>

Another approach is to use a class decorator, like this:

In [166]:
# Class decorator to apply constraints
def check_attributes(**kwargs):
    def decorate(cls):
        for key, value in kwargs.items():
            if isinstance(value, Descriptor):
                value.name = key
                setattr(cls, key, value)
            else:
                setattr(cls, key, value(key))
        return cls
    return decorate

In [166]:
# Example
@check_attributes(name=SizedString(size=8),
                  shares=UnsignedInteger,
                  price=UnsignedFloat)
class Stock(object):
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

Or, with an additional twist, a class decorator approach can also be used as a replacement for mixin classes, multiple inheritance, and tricky use of the super() function. Here is an alternative formulation of this recipe that uses class decorators:

In [171]:
# Base class. Uses a descriptor to set a value
class Descriptor(object):
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

In [172]:
# Decorator for applying type checking
def Typed(expected_type, cls=None):
    if cls is None:
        return lambda cls: Typed(expected_type, cls)
    super_set = cls.__set__
    def __set__(self, instance, value):
        if not isinstance(value, expected_type):
            raise TypeError('expected ' + str(expected_type))
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls

# Decorator for unsigned values
def Unsigned(cls):
    super_set = cls.__set__
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Expected >= 0')
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls

# Decorator for allowing sized values
def MaxSized(cls):
    super_init = cls.__init__
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super_init(self, name, **opts)
    cls.__init__ = __init__
    super_set = cls.__set__
    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super_set(self, instance, value)
    cls.__set__ = __set__
    return cls

In [173]:
# Specialized descriptors

@Typed(int)
class Integer(Descriptor):
    pass

@Unsigned
class UnsignedInteger(Integer):
    pass

@Typed(float)
class Float(Descriptor):
    pass

@Unsigned
class UnsignedFloat(Float):
    pass

@Typed(str)
class String(Descriptor):
    pass

@MaxSized
class SizedString(String):
    pass

## Problem 7: Calling a Method on an Object Given the Name As a String

You have the name of a method that you want to call on an object stored in a string and you want to execute the method.

In [175]:
import math

class Point(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'Point({!r:},{!r:})'.format(self.x, self.y)
    def distance(self, x, y):
        return math.hypot(self.x - x, self.y - y)

In [182]:
p = Point(2, 3)
p.distance(0,0)       # <- Usual call of method

3.605551275463989

In [180]:
d = getattr(p, 'distance')(0, 0)     # <- Calls p.distance(0, 0) as string
d

3.605551275463989

In [181]:
getattr(p, 'distance')(2, 3)     # Calls p.distance(0, 0)

0.0